In [1]:
%load_ext autoreload
%autoreload 2
#%matplotlib widget
#%matplotlib ipympl

#%reload_ext tensorboard
#%matplotlib qt

In [2]:
import os
from pathlib import Path
from datetime import datetime
import pandas as pd
import numpy as np
import scipy.io

import networkx as nx
from torch_geometric.utils import from_networkx

import matplotlib.pyplot as plt
import SimpleITK as sitk
from tqdm.auto import tqdm
import pickle, subprocess
import torch
import sklearn
import csv
import gc
import pydicom

import glob
from platipy.imaging import ImageVisualiser

from LN_malignancy_GNN.pytorch.dataset_class import DatasetGeneratorImage
from LN_malignancy_GNN.pytorch.run_model_torch import RunMalignancyModel
#from hnc_project import data_prep as dp
#from hnc_project import myshow
#from hnc_project import graph_making as gm
#from hnc_project.pytorch import dataset_class as dc
#from hnc_project.pytorch.run_model_torch import RunModel
#%matplotlib notebook
%matplotlib widget
plt.ion()
#import initial_ml as iml

In [3]:
data_dir = '../../data/ln_malignancy/'
graph_dir = '../../data/ln_malignancy/graph_staging'
edge_dir = '../../data/ln_malignancy/edge_staging'
patch_dir = '../../data/ln_malignancy/Extracted_Patches_v2'
primary_dir = '../../data/ln_malignancy/Processed_Data'
primary_subdir = 'primary_tumor'


data_path = Path(data_dir)
patch_path = Path(patch_dir)
graph_path = Path(graph_dir)
edge_path = Path(edge_dir)
primary_path = Path(primary_dir)

ct_path = patch_path.joinpath('ct')
feature_path = patch_path.joinpath('feature')
label_path = patch_path.joinpath('label')
seg_path = patch_path.joinpath('seg')


graph_path.mkdir(exist_ok=True, parents=True)
edge_path.mkdir(exist_ok=True, parents=True)


In [4]:
patients = pd.read_pickle(data_path.joinpath('features_and_labels_070924.pkl'))

In [5]:
patients

labels  pri_lat  ln_level  ln_lat  dist_to_pri  pri_location
patients nodes                                                              
5723648  1      [0, 1]      1.0       2.0     1.0    32.225766           1.0
         2      [1, 0]      1.0       2.0     1.0    55.780821           1.0
         3      [1, 0]      1.0       2.0     1.0    57.792733           1.0
         4      [0, 1]      1.0       2.0     1.0    32.225766           1.0
5759052  1      [1, 0]      1.0       2.0     1.0    61.532512           1.0
...                ...      ...       ...     ...          ...           ...
95427550 1      [1, 0]      0.0       3.0     0.0    77.175126           2.0
         2      [1, 0]      0.0       2.0     0.0    46.481179           2.0
         3      [0, 1]      0.0       2.0     0.0    39.623226           2.0
         4      [1, 0]      0.0       3.0     0.0    77.021101           2.0
         5      [1, 0]      0.0       3.0     0.0    84.581322           2.0

[1391 rows x 6 columns]

In [5]:
patients.index.levels[0]

Index(['5723648', '5759052', '70017472', '70034190', '70248556', '70488002',
       '70502204', '70524131', '70553974', '70587154',
       ...
       '94482638', '94504754', '94728956', '94747319', '94807664', '94942437',
       '95035464', '95058920', '95065073', '95427550'],
      dtype='object', name='patients', length=192)

In [5]:
from LN_project_repo.pytorch.gen_params_torch_cfg import *
test_data = DatasetGeneratorImage(config=model_config)

ModuleNotFoundError: No module named 'LN_project_repo'

In [22]:
fold = [0,1,2,3,4,5]
fold_pats = test_data.patients.iloc[fold].index
aug_pats = test_data.aug_patients.index
aug_fold_pats = [pat for pat in aug_pats if np.any([(fold_pat[0] in pat[0]) and (fold_pat[1] == pat[1]) for fold_pat in fold_pats])]
aug_fold_idx = test_data.aug_patients.index.get_indexer(aug_fold_pats) + len(test_data.patients.index.levels[0])
print(aug_fold_idx)

[192 193 194 195 196 197]


In [4]:
def rmdir(directory):
    directory = Path(directory)
    for item in directory.iterdir():
        if item.is_dir():
            rmdir(item)
        else:
            item.unlink()
    directory.rmdir()

### Find mask centroids

In [49]:
def find_centroid(mask, p):

    stats = sitk.LabelShapeStatisticsImageFilter()
    stats.Execute(mask)
    try:
        centroid_coords = stats.GetCentroid(1)
    except:
        print(f'Something is wrong with centroid calculation for patient: {p}')
    centroid_idx = mask.TransformPhysicalPointToIndex(centroid_coords)

    return np.asarray(centroid_idx, dtype=np.float64), np.asarray(centroid_coords, dtype=np.float64)

In [51]:
def tune_range(min_d, max_d, d, size_d, p):
    min_pad = 0
    max_pad = 0
    if min_d<0:
        min_pad = abs(min_d)
        min_d = 0
        #max_d = min_d + d
        #if max_d - size_d > 0:
        #    max_pad = max_d-size_d
            
        #assert (max_d<size_d), f"Cannot extract the patch with the shape {size_d} from the image with the shape {d} for patient {p}."
    
    if max_d>d:
        max_pad = max_d - d
        max_d = d
        #min_d = max_d - size_d
        #if min_d < 0:
        #    min_pad = abs(min_d)
            
        #assert (min_d>0), f"Cannot extract the patch with the shape {size_d} from the image with the shape {d} for patient {p}."

    return min_d, max_d, int(min_pad), int(max_pad)
patients_to_retry = [
    'HNSCC-01-0225',
    'HNSCC-01-0259',
    'HNSCC-01-0272',
    'HNSCC-01-0434',
]
physical_locations = {}
for p_dir in tqdm(list(resample_path.glob('*'))):
    p_str = p_dir.as_posix().split('/')[-1]
    print(p_str)
    #if p_str not in patients_to_retry: continue
    #try:
    #if p_str in patients_to_drop:
    #    print('skip ', p_str)
    #    continue
    patient_patch_path = patch_path.joinpath(p_str)
    patient_patch_path.mkdir(exist_ok=True, parents=True)
    physical_locations[p_str] = {}
    patch_size = np.array([50,50,20])
    for m in p_dir.glob('*.nii.gz'):
        print('-----------------')
        m_str = m.as_posix().split('/')[-1]
        if 'image' in m_str: continue
        #try:
        image = sitk.ReadImage(p_dir.joinpath('image.nii.gz'))
        mask = sitk.ReadImage(m)
        print(m_str)
        #crop the image to patch_size around the tumor center
        tumour_center, center_location = find_centroid(mask, p_str) # center of GTV
        size = patch_size
        min_coords = np.floor(tumour_center - size / 2).astype(np.int64)
        max_coords = np.floor(tumour_center + size / 2).astype(np.int64)
        min_x, min_y, min_z = min_coords
        max_x, max_y, max_z = max_coords
        (img_x, img_y, img_z)=image.GetSize()
        min_x, max_x, min_pad_x, max_pad_x = tune_range(min_x, max_x, img_x, size[0], p_str) 
        min_y, max_y, min_pad_y, max_pad_y = tune_range(min_y, max_y, img_y, size[1], p_str) 
        min_z, max_z, min_pad_z, max_pad_z = tune_range(min_z, max_z, img_z, size[2], p_str) 

        min_pad = int(max([min_pad_x, min_pad_y, min_pad_z]))
        max_pad = int(max([max_pad_x, max_pad_y, max_pad_z]))
        lpad = list([min_pad_x, min_pad_y, min_pad_z])
        upad = list([max_pad_x, max_pad_y, max_pad_z])
        #print(m_str)
        #print(lpad)
        #print(upad)
        print(image.GetSize())
        print(min_coords, max_coords)
        print(min_pad, max_pad)
        image = image[min_x:max_x, min_y:max_y, min_z:max_z]
        # window image intensities to [-500, 1000] HU range
        image = sitk.Clamp(image, sitk.sitkFloat32, -500, 500)
        mask = mask[min_x:max_x, min_y:max_y, min_z:max_z]
        print(image.GetSize())
        image = sitk.ConstantPad(image, lpad, upad, 0.0)
        mask = sitk.ConstantPad(mask, lpad, upad, 0.0)
        print(image.GetSize())
        sitk.WriteImage(image, patient_patch_path.joinpath(f"image_{m_str.replace('Struct_','')}"))
        sitk.WriteImage(mask, patient_patch_path.joinpath(m_str))
        physical_locations[p_str][m_str.replace('Struct_','').replace('.nii.gz','')] = center_location
        del(image)
        del(mask)
        #except:
        #    print(m)
        #    raise Exception('something went wrong...')
    
    #except:
    #    print(p_str)
        
with open(patch_path.joinpath('locations.pkl'), 'wb') as f:
    pickle.dump(physical_locations, f)
    f.close()

  0%|          | 0/3268 [00:00<?, ?it/s]

RADCURE-0005
-----------------
-----------------
Struct_GTVn_[L23].nii.gz
(250, 250, 152)
[118  80  31] [168 130  91]
0 0
(50, 50, 60)
(50, 50, 60)
-----------------
Struct_GTVn_[R2].nii.gz
(250, 250, 152)
[85 71 40] [135 121 100]
0 0
(50, 50, 60)
(50, 50, 60)
-----------------
Struct_GTVn_[R4].nii.gz
(250, 250, 152)
[92 83 10] [142 133  70]
0 0
(50, 50, 60)
(50, 50, 60)
-----------------
Struct_GTVn_[RRPN].nii.gz
(250, 250, 152)
[94 72 57] [144 122 117]
0 0
(50, 50, 60)
(50, 50, 60)
-----------------
Struct_GTVp.nii.gz
(250, 250, 152)
[101  69  45] [151 119 105]
0 0
(50, 50, 60)
(50, 50, 60)
RADCURE-0006
-----------------
-----------------
Struct_GTVp.nii.gz
(228, 228, 181)
[92 69 47] [142 119 107]
0 0
(50, 50, 60)
(50, 50, 60)
RADCURE-0007
-----------------
-----------------
Struct_GTVn_[L2AB].nii.gz
(250, 250, 172)
[127  73  62] [177 123 122]
0 0
(50, 50, 60)
(50, 50, 60)
-----------------
Struct_GTVp.nii.gz
(250, 250, 172)
[110  68  70] [160 118 130]
0 0
(50, 50, 60)
(50, 50, 60)
R

## Graph Making

In [4]:
patient_dict = {file.name.split('_')[0]: 
                   {'nodes': []}
                   #int(file.name.split('_')[-1].strip('.mat'))} 
                   for file in ct_path.glob('*.mat')}

for pat in patient_dict.keys():
    pat_files = ct_path.glob(f"{pat}_*.mat")
    patient_dict[pat]['nodes'] = sorted([int(file.name.split('_')[-1].strip('.mat')) for file in pat_files])

In [5]:
for pat in patient_dict.keys():
    pat_files = [f"{pat}_{node}.mat" for node in patient_dict[pat]['nodes']]
    patient_dict[pat]['features'] = [scipy.io.loadmat(feature_path.joinpath(f))['fea'].squeeze() for f in pat_files]
    patient_dict[pat]['labels'] = [scipy.io.loadmat(label_path.joinpath(f))['label'].squeeze() for f in pat_files]

patient_df = pd.DataFrame(patient_dict.values(), columns=['nodes', 'features', 'labels'], index=patient_dict.keys())
patient_df.index.name = 'patients'
patient_df = patient_df.explode(['nodes', 'features', 'labels'])
patient_df.set_index([patient_df.index, 'nodes'], inplace=True)
patient_df[['pri_lat', 'ln_level', 'ln_lat', 'dist_to_pri', 'pri_location']] = pd.DataFrame(patient_df['features'].tolist(), index=patient_df.index)
patient_df = patient_df.drop('features', axis=1)
patient_df.to_pickle(edge_path.joinpath('features_and_labels_110424.pkl'))
#patient_df['graph'] = None

In [11]:
patient_graph = {}
edge_dict = {}
for pat in patient_df.index.get_level_values('patients').unique():
    patient_graph[pat] = nx.DiGraph(directed=True)
    pat_df = patient_df.loc[pat]
    patient_graph[pat].add_nodes_from(patient_df.loc[pat].index)
    edge_dict[pat] = []
    for _, group_df in pat_df.groupby(['ln_lat']):
        if len(group_df.index) < 2:
            edge_dict[pat].extend([(node, node) for node in group_df.index])
            continue
        for node in group_df.index:
            edge_dict[pat].extend([(node, jnode) for jnode in group_df.index 
                                           if (node != jnode) 
                                           and (group_df.loc[node, 'ln_level'] <= 
                                                group_df.loc[jnode, 'ln_level'])
                                           and np.abs(group_df.loc[node, 'ln_level']
                                                - group_df.loc[jnode, 'ln_level']) <= 1])
    patient_graph[pat].add_edges_from(edge_dict[pat])
    patient_graph[pat] = patient_graph[pat].to_undirected()
#graph_df = pd.DataFrame(data=patient_graph.values(), index=patient_graph.keys(), columns=["graph"])
#graph_df = pd.DataFrame(data=patient_graph.values(), index=patient_graph.keys())
#patient_df['graph'] = patient_df['graph'].map(patient_graph)
#for idx_node in range(patient_dict['70248556']['nodes']):
#    tpgraph.add_node(idx_node

In [12]:
with open(edge_path.joinpath('proto_graphs_undirected_111524.pkl'), 'wb') as f:
    pickle.dump(patient_graph, f)
    f.close()

In [12]:
#Complete graph
patient_graph = {}
edge_dict = {}
for pat in patient_df.index.get_level_values('patients').unique():
    patient_graph[pat] = nx.DiGraph(directed=True)
    pat_df = patient_df.loc[pat]
    patient_graph[pat].add_nodes_from(patient_df.loc[pat].index)
    edge_dict[pat] = []
    for _, group_df in pat_df.groupby(['ln_lat']):
        for node in group_df.index:
            edge_dict[pat].extend([(node, jnode) for jnode in group_df.index 
                                           if (node != jnode)]) 
    patient_graph[pat].add_edges_from(edge_dict[pat])
#graph_df = pd.DataFrame(data=patient_graph.values(), index=patient_graph.keys(), columns=["graph"])
#graph_df = pd.DataFrame(data=patient_graph.values(), index=patient_graph.keys())
#patient_df['graph'] = patient_df['graph'].map(patient_graph)
#for idx_node in range(patient_dict['70248556']['nodes']):
#    tpgraph.add_node(idx_node

In [13]:
with open(edge_path.joinpath('proto_graphs_complete_110424.pkl'), 'wb') as f:
    pickle.dump(patient_graph, f)
    f.close()

In [6]:
patient_graph = {}
edge_dict = {}
for pat in patient_df.index.get_level_values('patients').unique():
    patient_graph[pat] = nx.DiGraph(directed=True)
    pat_df = patient_df.loc[pat]
    patient_graph[pat].add_nodes_from(patient_df.loc[pat].index)
    edge_dict[pat] = []
    for _, group_df in pat_df.groupby(['ln_lat']):
        #if len(group_df.index) < 2:
        #    edge_dict[pat].extend([(node, node) for node in group_df.index])
        #    continue
        for node in group_df.index:
            edge_dict[pat].extend([(node, jnode) for jnode in group_df.index 
                                           if (node != jnode) 
                                           and (group_df.loc[node, 'ln_level'] <= 
                                                group_df.loc[jnode, 'ln_level'])
                                           and np.abs(group_df.loc[node, 'ln_level']
                                                - group_df.loc[jnode, 'ln_level']) <= 1])
            edge_dict[pat].extend([(jnode, node) for jnode in group_df.index 
                                           if (node != jnode) 
                                           and (group_df.loc[node, 'ln_level'] <= 
                                                group_df.loc[jnode, 'ln_level'])
                                           and np.abs(group_df.loc[node, 'ln_level']
                                                - group_df.loc[jnode, 'ln_level']) <= 1])
    patient_graph[pat].add_edges_from(edge_dict[pat])
    patient_graph[pat].add_node(0)
    for node in patient_graph[pat].nodes:
        patient_graph[pat].add_edge(0, node)
    #patient_graph[pat] = patient_graph[pat].to_undirected()
#graph_df = pd.DataFrame(data=patient_graph.values(), index=patient_graph.keys(), columns=["graph"])
#graph_df = pd.DataFrame(data=patient_graph.values(), index=patient_graph.keys())
#patient_df['graph'] = patient_df['graph'].map(patient_graph)
#for idx_node in range(patient_dict['70248556']['nodes']):
#    tpgraph.add_node(idx_node

In [7]:
with open(edge_path.joinpath('proto_graphs_directed_primary_112724.pkl'), 'wb') as f:
    pickle.dump(patient_graph, f)
    f.close()

In [8]:
#patient_df.to_pickle(data_path.joinpath('features_and_labels_092524.pkl'))

In [12]:
patient_graph['5723648'].nodes

NodeView((1, 2, 3, 4))

In [9]:
26 + 24//2 + (24//2+24%2)

50

In [10]:
test = torch.load(os.path.join('./logs/training_chosen_split_no_graph_lncnn_noedgedim_no_pretrain_nofreeze_copy_attempt_ch2_class2_dp2_weight1_10rot_balance_feat256_nested_clinical_v15', 'top_models_fold_0/model_epoch=50_val_loss=0.84_val_auc=0.98_val_m=0.93.ckpt'))

In [3]:
from LN_malignancy_GNN.pytorch.gen_params_torch_cfg import model_config
model_config['batch_size'] = 1
model_config['dropout'] = 0.2
#model_config['ext_dropout'] = 0.0
model_config['n_epochs'] = 100
model_config['learning_rate'] = 1e-4
model_config['lr_factor'] = 0.1
model_config['lr_patience'] = 10
#model_config['l2_reg'] = 1e-3
model_config['n_classes'] = 2
#model_config['data_type'] = 'image'
#model_config['model_name'] = 'DeepGCN'
##model_config['model_name'] = 'ClinicalGatedGCN'
#model_config['model_name'] = 'ResNet50'
##model_config['model_name'] = 'GraphAgResNet'
#model_config['model_name'] = 'EmptyNetwork'
model_config['model_name'] = 'GatedGCN'
##model_config['extractor_name'] = 'None'
model_config['extractor_name'] = 'LNCNN'
model_config['extractor_channels'] = 256
model_config['freeze_extractor'] = False
model_config['pretrain'] = False
##### hidden chanels needs to be set to extractor output when
##### using the EmptyNetwork, since it is just a pass through network
model_config['n_hidden_channels'] = 2
model_config['gnn_out_channels'] = 2
##model_config['extractor_name'] = 'DenseNet'
#model_config['with_edge_attr'] = False
model_config['use_clinical'] = True
model_config['n_clinical'] = 5
model_config['include_primary'] = False
model_config['edge_dim'] = None
#model_config['scaling_type'] = 'ZScore'
#model_config['cross_val'] = True
#model_config['nested_cross_val'] = False
#model_config['num_deep_layers'] = 28
#model_config['augment'] = True
#model_config['augments'] = [
#                            'rotation',
#                           ]
model_config['n_rotations'] = 10
model_config['balance_classes'] = True
model_config['region']: 'ROI'
model_config['n_in_channels'] = 2
model_config['preset_folds'] = True
model_config['class_weight'] = 1.
model_config['alternate_splitting'] = True
#model_config['n_extracted_features'] = 2048
#model_config['n_hidden_channels'] = 64
#model_config['log_dir'] = 'training_resnet18_augment_rot1_balance_mask_v2'
#model_config['log_dir'] = 'training_chosen_split_graph_lncnn_noedgedim_no_pretrain_nofreeze_copy_attempt_ch2_class2_dp2_weight1_10rot_balance_feat256_feat8_4_nested_v13'
model_config['log_dir'] = 'training_chosen_split_graph_lncnn_noedgedim_no_pretrain_nofreeze_copy_attempt_ch2_class2_dp2_weight1_10rot_balance_feat256_feat2_2_nested_clinical_v15'
model_config['data_version'] = '505020_rot10_30_balance_class2_v4'
model_config['patch_dir'] = 'Extracted_Patches_v2'
#model_config['patient_feature_file'] = 'features_and_labels_092524.pkl'
#model_config['edge_file'] = 'proto_graphs_092524.pkl'
model_config['patient_feature_file'] = 'features_and_labels_070924.pkl'
model_config['edge_file'] = 'proto_graphs_070924.pkl'

## v1 is 29, v2 is 32, v3 is 22, v4 is 28, v5 is 8, v7 is 28, v8 is 22, v9 is 18
#model_config['preset_folds'] = False
#model_config['preset_fold_file'] = 'preset_folds_wGTVp.pkl'


In [4]:
model = RunMalignancyModel(model_config)
model.set_model()
#model.set_data()

Seed set to 42


logs are located at: logs/training_chosen_split_graph_lncnn_noedgedim_no_pretrain_nofreeze_copy_attempt_ch2_class2_dp2_weight1_10rot_balance_feat256_feat2_2_nested_clinical_v15
remember to set the data


In [15]:
model.load_fold_weights(0)

RuntimeError: Error(s) in loading state_dict for CNN_GNN:
	size mismatch for classify.classify.weight: copying a param with shape torch.Size([2, 261]) from checkpoint, the shape in current model is torch.Size([2, 7]).

In [9]:
model.model.extractor.cn1.weight

Parameter containing:
tensor([[[[[ 5.5564e-04,  1.8504e-02, -1.3436e-02, -6.9613e-03,  1.0518e-02],
           [-1.5840e-02,  2.4387e-03, -1.8841e-02,  2.4897e-02, -1.7806e-02],
           [-1.6805e-02,  1.4082e-02,  2.6403e-02, -8.4943e-04,  1.6134e-02],
           [ 4.1762e-03, -2.5131e-02, -2.7882e-03, -1.8531e-02,  2.3437e-03],
           [ 1.8573e-02,  1.8258e-02,  1.4550e-02,  2.5026e-02,  1.0619e-02]],

          [[-3.8807e-03, -1.5323e-02, -7.5380e-03, -1.2416e-03, -4.6496e-03],
           [-2.5717e-02,  2.3165e-02,  1.4284e-02,  1.4580e-02, -1.5659e-02],
           [ 2.5692e-02, -1.0558e-02,  2.1045e-02,  6.9200e-03,  1.0187e-02],
           [-1.4351e-02,  8.4752e-03, -1.9642e-02,  2.8391e-04, -2.2459e-03],
           [ 1.7015e-03, -2.2281e-02, -1.9099e-02,  1.0416e-02,  7.5144e-03]],

          [[-1.4828e-03,  4.3564e-03, -1.0904e-02,  1.7742e-03, -2.5054e-02],
           [-2.1664e-02, -2.2080e-02,  2.1910e-02, -1.0528e-02,  2.6048e-02],
           [-3.8238e-03,  1.0425e-02, 

In [12]:
test['state_dict']['extractor.cn1.weight']

tensor([[[[[ 9.8397e-03, -2.7729e-02, -2.3817e-02,  1.1563e-02,  7.8626e-03],
           [-3.1207e-04,  1.3569e-02, -1.1679e-04,  2.2460e-02, -3.0435e-03],
           [ 3.2287e-02,  2.0911e-02, -3.5598e-02,  1.0956e-02,  1.0128e-02],
           [ 3.1470e-03,  1.8336e-02, -7.6509e-03, -5.1222e-02,  1.6745e-02],
           [-3.4965e-02, -4.6510e-02, -1.2565e-02, -2.4818e-02, -2.4306e-03]],

          [[-1.3464e-02, -1.5448e-02, -2.5962e-02, -3.0139e-03,  2.3347e-02],
           [ 3.4323e-02,  2.7693e-02,  3.6517e-03, -1.3262e-02, -1.5556e-02],
           [ 5.0655e-02,  2.5907e-02, -1.6900e-02, -5.3271e-03, -1.2216e-02],
           [ 7.3410e-03,  4.3055e-02,  7.1228e-03, -4.2835e-02,  5.7552e-03],
           [-2.4178e-02, -1.6051e-02, -2.1194e-02, -4.6552e-02, -2.4038e-02]],

          [[ 1.1578e-02, -3.9682e-03, -2.6734e-04, -8.8312e-03, -1.7236e-02],
           [ 1.6533e-02, -5.9660e-03, -2.2628e-02,  2.7812e-03, -2.5630e-02],
           [ 3.3020e-02,  3.9502e-02,  1.3012e-02, -4.7392e-

In [37]:
input_folds = pd.read_pickle('../../data/ln_malignancy/preset_folds_malignancy_liyuan_091024.pkl')

In [41]:
len(input_folds[1])

37